# 동사무소 -> 집
---
1. 트럭
    - 속도: 30km/h
    - 대기 시간: 3m
    - osrm을 통한 경로 추출
2. 로봇
    - 여러 대 배치
    - 속도: 5.76km/h
    - ortools로 각 로봇마다의 경로 추출?(지나가는 집들 정해주기) 
    - 경로를 추출 했으면 osrm으로 경로 시각화

- 트럭과 로봇 전체 이용 가능한 OD 데이터 생성    
    - O: 동사무소 지점, D: 각 집들 지점


# 기본 데이터프레임

동사무소를 시작으로 배달해야하는 집들을 모은 데이터

In [14]:
import pandas as pd
from shapely.geometry import Point

In [15]:
# 변수 설정
## O 데이터
O = ['동사무소', '동사무소', '동사무소']
O_point = [[127.133374, 37.455009], [127.133374, 37.455009], [127.133374, 37.455009]]
O_time = [600, 600, 600]

## D 데이터
D = ['집1', '집2', '집3']
D_point = [[127.133923, 37.453348], [127.130112, 37.452589], [127.127384, 37.45091]]

# 좌표를 WKT 형식으로 변환
O_point_wkt = [f"POINT ({coord[0]} {coord[1]})" for coord in O_point]
D_point_wkt = [f"POINT ({coord[0]} {coord[1]})" for coord in D_point]

# 데이터프레임 구성
OD_data = pd.DataFrame({
    'O': O,
    'O_point': O_point_wkt,
    'O_time': O_time,
    'D': D,
    'D_point': D_point_wkt
})

# 문자열을 Point 객체로 변환하는 함수
def convert_to_point(point_str):
    # 'POINT (127.127384 37.45091)' -> [127.127384, 37.45091]
    coords = point_str.replace("POINT (", "").replace(")", "").split()
    return Point(float(coords[0]), float(coords[1]))

# 'start_point'와 'end_point'를 Point 객체로 변환
OD_data['O_point'] = OD_data['O_point'].apply(convert_to_point)
OD_data['D_point'] = OD_data['D_point'].apply(convert_to_point)

In [16]:
OD_data

,O,O_point,O_time,D,D_point
0,동사무소,POINT (127.133374 37.455009),600,집1,POINT (127.133923 37.453348)
1,동사무소,POINT (127.133374 37.455009),600,집2,POINT (127.130112 37.452589)
2,동사무소,POINT (127.133374 37.455009),600,집3,POINT (127.127384 37.45091)


# 트럭

트럭 데이터프레임 구성
- O: 동사무소 지점
- D: 각 집들의 지점
- 속도: 30km/h(골목이므로 어린이 보호구역을 기준으로 가정)
- 대기 시간(기사가 내려서 배달하는 시간): 3min으로 가정
- 출발 시간: 출발 시간을 설정해서 최종 duration 계산, 시각화에 사용
- 도착 시간: 거리와 속도 계산을 통해 최종 출발 시간 + duration으로 구하기
- duration: 총 걸린 시간(이동 시간 + 대기 시간)
- distance: 총 걸린 거리

In [17]:
# 트럭 데이터프레임 구성하기

# 변수 설정하기
## 트럭 데이터
speed = [30, 30, 30]
wait_time = [0, 0, 0]
arrive_time = [0, 0, 0]
duration = [0, 0, 0]
distance = [0, 0, 0]

# 데이터프레임 구성하기
OD_data_truck = OD_data.copy()
OD_data_truck['speed(km/h)'] = speed
OD_data_truck['wait_time'] = wait_time
OD_data_truck['arrive_time'] = arrive_time
OD_data_truck['duration'] = duration
OD_data_truck['distance(단위)'] = distance

In [18]:
OD_data_truck

,O,O_point,O_time,D,D_point,speed(km/h),wait_time,arrive_time,duration,distance(단위)
0,동사무소,POINT (127.133374 37.455009),600,집1,POINT (127.133923 37.453348),30,0,0,0,0
1,동사무소,POINT (127.133374 37.455009),600,집2,POINT (127.130112 37.452589),30,0,0,0,0
2,동사무소,POINT (127.133374 37.455009),600,집3,POINT (127.127384 37.45091),30,0,0,0,0


In [1]:
# 라이브러리 불러오기
import numpy as np
import itertools
import requests
import polyline
import json
import os
import math

import random as rd
import pandas as pd
import geopandas as gpd

from datetime import datetime, timedelta

from shapely.geometry import Point

from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

import warnings 

warnings.filterwarnings('ignore')

In [2]:
# 좌표 거리 생성 함수
## 직선 거리 계산 함수
## 거리 계산할 때 경로를 모르는 경우 사용
def calculate_straight_distance(lat1, lon1, lat2, lon2):
    '''
    좌표 거리 생성 함수 
    - 직선 거리 계산 
    - 경로를 모르는 경우 두 지점 간의 대략적인 직선 거리를 계산할 때 사용

    입력값:
        lat1: 출발지 위도 (float)
        lon1: 출발지 경도 (float)
        lat2: 도착지 위도 (float)
        lon2: 도착지 경도 (float)

    출력값:
        두 지점 간의 직선 거리 (킬로미터 단위, float)
    '''
# 직선 거리 계산
# 입력값: 출발지 위도, 경도 / 도착지 위도, 경도
    # 지구 반경 (킬로미터 단위)
    km_constant = 3959* 1.609344
    
    # 위도와 경도를 라디안으로 변환
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    
    # 위도 및 경도 차이 계산
    dlat = lat2 - lat1 # 도착지 위도 - 출발지 위도
    dlon = lon2 - lon1 # 도착지 경도 - 출발지 경도

    # Haversine 공식 계산
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    # 거리 계산 (킬로미터 단위)
    km = km_constant * c
    
    return km # 출발지-도착지 사이의 직선 거리 반환

In [3]:
# trips 데이터 생성 함수
#### osrm 페키지로 경로 추출
# 입력으로 받은 출발지와 목적지 좌표를 이용하여 경로 정보를 가져오는 함수
def get_res(point, mode = 'foot'): # 도보 이용

   # point: 출발지와 목적지 좌표 정보를 포함하는 리스트[lat1, lon1, lat2, lon2]
   # mode: 이동 수단 (기본값: 도보)

   status = 'defined'

   # 요청을 재시도할 수 있도록 세션 객체 생성 및 설정
   session = requests.Session()
   retry = Retry(connect=3, backoff_factor=0.5)
   adapter = HTTPAdapter(max_retries=retry)
   session.mount('http://', adapter)
   session.mount('https://', adapter)

   #### url 생성 코드
   # 전체 경로 정보를 요청
   overview = '?overview=full'
   # lon, lat, lon, lat 형식의 출발지 목적지 좌표
   loc = f"{point[0]},{point[1]};{point[2]},{point[3]}"
   # 보행경로 url
   url = f'http://router.project-osrm.org/route/v1/{mode}/'
   # 경로 정보 요청
   r = session.get(url + loc + overview) 
   
   # 만약 경로가 안뜰 때 대체 결과 생성
   ## 직선 거리 구하기
   if r.status_code!= 200:
      
      status = 'undefined'
      
       # 직선 거리 계산
      distance = calculate_straight_distance(point[1], point[0], point[3], point[2]) * 1000
      
      # 경로 정보 생성 (출발지와 목적지 좌표만 포함)
      route = [[point[0], point[1]], [point[2], point[3]]]

      # 소요 시간 및 타임스탬프 계산 (가정: 보행 속도 10km/h)
      speed_km = 5#km
      speed = (speed_km * 1000/60)      
      duration = distance/speed
      
      timestamp = [0, duration]

      result = {'route': route, 'timestamp': timestamp, 'duration': duration, 'distance' : distance}
   
      return result, status
   
   # 경로 정보를 성공적으로 가져온 경우, JSON 응답을 반환
   res = r.json()   
   return res, status

In [4]:
# 경로를 가는데 걸리는 시간과 거리 추출 함수
def extract_duration_distance(res, speed_kmh):
   # get_res함수에서 추출된 데이터에서 시간과 거리 뽑기
   # 입력값: res(get_res함수에서 추출된 데이터), 속도 (km/h)
   
   distance = res['routes'][0]['distance']
   # JSON 응답에서 첫 번째 경로의 거리 값을 추출
   # m 단위로 거리 추출
   # duration = res['routes'][0]['duration']/(60)  # 분 단위로 변환
   
   # 속도로 시간 계산
   speed_kmh = speed_kmh  # km/h
   speed_mps = speed_kmh * 1000 / 3600  # 속도를 m/s로 변환
   duration = distance / speed_mps / 60  # 분 단위로 변환
   
   return duration, distance # 소요 시간, 거리 반환

# 경로 추출 함수
def extract_route(res):
    # 입력값: res(get_res함수에서 추출된 데이터)
   
    # get_res함수에서 추출된 데이터에서 경로 뽑기
    # 경로가 인코딩 되어 있기 때문에 아래 함수를 써서 디코딩해주어야지 위경도로 이루어진 경로가 나옴
    route = polyline.decode(res['routes'][0]['geometry'])
    
    # 사용할 형식에 맞춰 위경도 좌표의 위치를 바꿔주는 것
    route = list(map(lambda data: [data[1],data[0]] ,route))
    # data: [위도, 경도] 형식의 좌표 쌍 -> [경도, 위도] 형식의 좌표 쌍

    return route # [[127.0, 37.0], [127.1, 37.1], ...] 형식의 경로 반환

In [5]:
# 총 걸리는 시간을 경로의 거리 기준으로 쪼개주는 함수
def extract_timestamp(route, duration):
    '''
    경로의 각 구간 거리 비율에 따라 예상 도착 시간을 계산하는 함수

    입력값:
        route: 경로 정보 (위도와 경도로 구성된 리스트)
        duration: 총 소요 시간 (분 단위, float)
    
    출력값:
        timestamp: 각 지점의 예상 도착 시간 리스트 (분 단위)
    '''
    # 입력값: route(경로 정보), duration(총 소요 시간)
    
    # 리스트를 numpy이 배열로 변경
    rt = np.array(route)
    # 리스트를 수평 기준으로 합치기
    rt = np.hstack([rt[:-1,:], rt[1:,:]])
    # [출발점_lat, 출발점_lon, 도착점_lat, 도착점_lon] 형식으로 변환

    # 각각 직선거리 추출(리스트 형태)
    per = calculate_straight_distance(rt[:,1], rt[:,0], rt[:,3], rt[:,2])
    # 출발점 경도, 출발점 위도, 도착점 경도, 도착점 위도를 이용하여 직선거리 계산

    # 각각의 직선거리를 전체 직선거리의 합으로 나누기
    per = per / np.sum(per)
    # 각 구간의 직선 거리 비율 계산
    # 전체 경로에서 해당 구간이 차지하는 비율 계산

    # 계산된 비율을 기반으로 각 지점 도착 예상 시간 계산
    timestamp = per * duration
    ## 각 구간의 비율에 전체 소요 시간을 곱해 각 구간의 소요 시간 계산
    
    # 각 구간의 소요 시간을 누적하여 더하기
    timestamp = np.hstack([np.array([0]),timestamp])

    # 타임스탬프 배열의 누적 합을 계산하여 각 지점의 예상 도착 시간을 생성
    timestamp = list(itertools.accumulate(timestamp)) 
    
    return timestamp # 각 지점의 예상 도착 시간 리스트 e.g. [0, 10, 20, 30, ...] 반환

In [6]:
from functools import partial

# 모든 함수를 한번에 실행하는 코드(trips 데이터의 형태로 저장)
def osrm_routing_machine(O, D, mode, speed_kmh):
   # 입력값: 출발지 좌표, 목적지 좌표, 이동 수단, 속도

   # osrm 데이터 생성
   osrm_base, status = get_res([O.x, O.y, D.x, D.y], mode)
   
   # osrm 데이터가 생성 됬으면 진행
   if status == 'defined':
      # 거리 및 걸리는 시간 추출
      duration, distance = extract_duration_distance(osrm_base, speed_kmh)
      # 경로 추출
      route = extract_route(osrm_base)
      # timestamp 생성
      timestamp = extract_timestamp(route, duration)
      # 결과 저장
      result = {'route': route, 'timestamp': timestamp, 'duration': duration, 'distance' : distance}
      
      return result
   else: 
      return osrm_base # 경로 데이터 없을 때는 직선거리 데이터 반환
   
# OD_data 한쌍일 때 osrm_routing_machine작동함수
def osrm_routing_machine_multiprocess(OD, mode, speed_kmh):
   O, D = OD
   result = osrm_routing_machine(O, D, mode, speed_kmh)
   return result
# OD_data 데이터가 리스트쌍 일때의 osrm_routing_machine 작동함수
def osrm_routing_machine_multiprocess_all(OD_data, mode, speed_kmh):
    results = list(map(partial(osrm_routing_machine_multiprocess, mode = mode, speed_kmh=speed_kmh), OD_data))
    return results

In [7]:
# 출발시간 기반으로 탑승시간 계산 함수
def calculate_boarding_time(start_times):
    '''
    출발시간을 기반으로 탑승 시간을 계산하는 함수
    - 버스는 10분 간격으로 출발한다고 가정

    입력값:
        start_times: 출발 시간 리스트 (분 단위)

    출력값:
        boarding_times: 각 출발 시간에 대응하는 탑승 시간 리스트 (분 단위)
    '''
    boarding_times = []
    for start_time in start_times:
        # 버스는 10분 간격으로 출발한다고 가정
        boarding_time = math.ceil(start_time / 10) * 10
        boarding_times.append(boarding_time)
    return boarding_times

# 데이터프레임에서 OD 데이터와 출발시간, 탑승시간 데이터 추출 함수
def extract_od_and_start_time(df):
    '''
    데이터프레임에서 OD 데이터와 출발 시간, 탑승 시간 데이터를 추출하는 함수

    입력값:
        df: OD 데이터가 포함된 데이터프레임
            - 컬럼: start_point (출발지 위경도), end_point (도착지 위경도), start_time, boarding_time

    출력값:
        od_data: OD 데이터 리스트 ([[출발지 위경도, 도착지 위경도], ...])
        start_time: 출발 시간 리스트
        boarding_time: 탑승 시간 리스트
    '''
    # OD 데이터를 추출 (출발점과 도착점의 위경도)
    od_data = [[row['start_point'], row['end_point']] for _, row in df.iterrows()]
    
    # 출발시간과 탑승시간 데이터를 리스트로 추출
    start_time = df['start_time'].tolist()
    boarding_time = df['boarding_time'].tolist()
    
    return od_data, start_time, boarding_time

In [8]:
# 경로 생성 함수들: get_res, extract_duration_distance, extract_route, extract_timestamp 사용
# 그 외 보조 함수들: calculate_straight_distance, calculate_boarding_time, osrm_routing_machine 등 사용

def calculate_route_with_stops(start_point, stops, mode, speed_kmh, wait_time=3):
    """
    OSRM을 사용해 시작 지점에서 특정 정류장들을 거쳐 다시 시작 지점으로 돌아오는 경로를 계산.

    Args:
        start_point (Point): 시작 지점 (Shapely Point 객체)
        stops (list): 방문해야 할 정류장 리스트 (Shapely Point 객체 리스트)
        mode (str): 이동 모드 (e.g., 'car', 'foot', 'bike')
        speed_kmh (float): 차량 속도 (km/h)
        wait_time (int): 각 정류장에서 대기 시간 (분 단위)

    Returns:
        dict: 경로, 시간, 거리 정보가 포함된 딕셔너리
    """
    # 경로 계산 결과 초기화
    total_route = []
    total_duration = 0
    total_distance = 0
    timestamps = []

    # 경유지를 포함한 순환 경로: 시작 지점 -> 각 정류장 -> 시작 지점
    all_points = [start_point] + stops + [start_point]

    # 각 구간에 대해 경로 계산
    for i in range(len(all_points) - 1):
        O, D = all_points[i], all_points[i + 1]
        
        # OSRM 경로 계산
        result = osrm_routing_machine(O, D, mode, speed_kmh)
        segment_route = result['route']
        segment_duration = result['duration']
        segment_distance = result['distance']
        
        # 경로, 시간, 거리 누적
        total_route.extend(segment_route[:-1])  # 마지막 점은 중복 방지
        total_duration += segment_duration
        total_distance += segment_distance

        # 대기 시간 추가 (정류장에서만)
        if i > 0 and i < len(all_points) - 2:  # 중간 정류장
            total_duration += wait_time

        # 타임스탬프 갱신
        timestamps.append(total_duration)

    # 최종 경로의 마지막 지점 추가
    total_route.append(segment_route[-1])

    # 결과 반환
    return {
        "route": total_route,
        "timestamps": timestamps,
        "total_duration": total_duration,
        "total_distance": total_distance,
    }

# 예시 데이터
start = Point(127.133374, 37.455009)  # 시작 지점
stops = [
    Point(127.133923, 37.453348),  # 정류장 1
    Point(127.130112, 37.452589),  # 정류장 2
    Point(127.127384, 37.450910)   # 정류장 3
]

# 경로 생성 실행
mode = 'car'  # 차량 이동
speed_kmh = 50  # 차량 속도 (km/h)
wait_time = 3  # 정류장 대기 시간 (분 단위)

result = calculate_route_with_stops(start, stops, mode, speed_kmh, wait_time)

# 결과 출력
print("Route:", result["route"])
print("Timestamps:", result["timestamps"])
print("Total Duration (min):", result["total_duration"])
print("Total Distance (m):", result["total_distance"])


Route: [[127.13322, 37.455], [127.13325, 37.45481], [127.13336, 37.45455], [127.13344, 37.45439], [127.13357, 37.45427], [127.13369, 37.45418], [127.13376, 37.45407], [127.13379, 37.45387], [127.13383, 37.45373], [127.13402, 37.45345], [127.13391, 37.45337], [127.1339, 37.45337], [127.13375, 37.45324], [127.13357, 37.45305], [127.13337, 37.45297], [127.1332, 37.45291], [127.13312, 37.45287], [127.13217, 37.45251], [127.13208, 37.45247], [127.13203, 37.45245], [127.13198, 37.45243], [127.13192, 37.45239], [127.13187, 37.45236], [127.13173, 37.45232], [127.13161, 37.45229], [127.13152, 37.45226], [127.13144, 37.45222], [127.13137, 37.45216], [127.13131, 37.45206], [127.13125, 37.45197], [127.13121, 37.45196], [127.13117, 37.45196], [127.13113, 37.45196], [127.1311, 37.45198], [127.13106, 37.45202], [127.13067, 37.45254], [127.13063, 37.45258], [127.1306, 37.45259], [127.13057, 37.4526], [127.13049, 37.45259], [127.13035, 37.45255], [127.13019, 37.45249], [127.13017, 37.45249], [127.13004

In [11]:
result

{'route': [[127.13322, 37.455],
  [127.13325, 37.45481],
  [127.13336, 37.45455],
  [127.13344, 37.45439],
  [127.13357, 37.45427],
  [127.13369, 37.45418],
  [127.13376, 37.45407],
  [127.13379, 37.45387],
  [127.13383, 37.45373],
  [127.13402, 37.45345],
  [127.13391, 37.45337],
  [127.1339, 37.45337],
  [127.13375, 37.45324],
  [127.13357, 37.45305],
  [127.13337, 37.45297],
  [127.1332, 37.45291],
  [127.13312, 37.45287],
  [127.13217, 37.45251],
  [127.13208, 37.45247],
  [127.13203, 37.45245],
  [127.13198, 37.45243],
  [127.13192, 37.45239],
  [127.13187, 37.45236],
  [127.13173, 37.45232],
  [127.13161, 37.45229],
  [127.13152, 37.45226],
  [127.13144, 37.45222],
  [127.13137, 37.45216],
  [127.13131, 37.45206],
  [127.13125, 37.45197],
  [127.13121, 37.45196],
  [127.13117, 37.45196],
  [127.13113, 37.45196],
  [127.1311, 37.45198],
  [127.13106, 37.45202],
  [127.13067, 37.45254],
  [127.13063, 37.45258],
  [127.1306, 37.45259],
  [127.13057, 37.4526],
  [127.13049, 37.45259]

In [12]:
import folium

def visualize_route_with_lines(result, start_point, stops):
    """
    OSRM 경로 데이터를 folium으로 시각화하고 경로를 선으로 표현하는 함수.

    Args:
        result (dict): OSRM 경로 계산 결과
        start_point (Point): 시작 지점 (Shapely Point 객체)
        stops (list): 방문해야 할 정류장 리스트 (Shapely Point 객체 리스트)
    """
    # 지도 초기화 (시작 지점을 중심으로 설정)
    m = folium.Map(location=[start_point.y, start_point.x], zoom_start=15)

    # 경로를 선으로 추가
    if result["route"]:
        # 경로 좌표를 [latitude, longitude] 형식으로 변환
        route_latlon = [[coord[1], coord[0]] for coord in result["route"]]
        folium.PolyLine(
            locations=route_latlon,  # 변환된 경로 좌표 리스트
            color="blue",
            weight=5,
            opacity=0.7,
            tooltip=f"Total Distance: {result['total_distance']:.2f} m, Total Duration: {result['total_duration']:.2f} min"
        ).add_to(m)

    # 시작 지점 표시
    folium.Marker(
        [start_point.y, start_point.x],
        icon=folium.Icon(color="green"),
        tooltip="Start Point"
    ).add_to(m)

    # 정류장 표시
    for i, stop in enumerate(stops, start=1):
        folium.Marker(
            [stop.y, stop.x],
            icon=folium.Icon(color="red", icon="info-sign"),
            tooltip=f"Stop {i}"
        ).add_to(m)

    # 마지막 지점 표시 (돌아온 시작 지점)
    folium.Marker(
        [start_point.y, start_point.x],
        icon=folium.Icon(color="blue", icon="flag"),
        tooltip="End Point"
    ).add_to(m)

    # 지도 반환
    return m

# 시각화 실행
map_visualization = visualize_route_with_lines(result, start, stops)

# 지도 저장 및 출력
map_visualization.save("route_visualization_with_lines.html")
map_visualization

# 로봇
로봇 데이터프레임 구성
- O: 동사무소 지점
- D: 각 집들의 지점
- 속도: 5.76km/h - 법적 최고 시간을 기준으로 함(정확히 찾아보기)
- 출발 시간: 출발 시간을 설정해서 최종 duration 계산, 시각화에 사용
- 도착 시간: 거리와 속도 계산을 통해 최종 출발 시간 + duration으로 구하기
- duration: 총 걸린 시간(가장 마지막으로 동사무소로 회수되는 시간)
- distance: 총 걸린 거리(로봇들의 평균 거리)

\* 몇 대 할 건지는 경우의 수를 두고 테스트

In [19]:
# 로봇 데이터프레임 구성하기

# 변수 설정하기
## 트럭 데이터
speed = [5.76, 5.76, 5.76]
arrive_time = [0, 0, 0]
duration = [0, 0, 0]
distance = [0, 0, 0]

# 데이터프레임 구성하기
OD_data_robot = OD_data.copy()
OD_data_robot['speed(km/h)'] = speed
OD_data_robot['arrive_time'] = arrive_time
OD_data_robot['duration'] = duration
OD_data_robot['distance(단위)'] = distance

In [20]:
OD_data_robot

,O,O_point,O_time,D,D_point,speed(km/h),arrive_time,duration,distance(단위)
0,동사무소,POINT (127.133374 37.455009),600,집1,POINT (127.133923 37.453348),5.76,0,0,0
1,동사무소,POINT (127.133374 37.455009),600,집2,POINT (127.130112 37.452589),5.76,0,0,0
2,동사무소,POINT (127.133374 37.455009),600,집3,POINT (127.127384 37.45091),5.76,0,0,0
